In [1]:
import pandas as pd

In [2]:
# !conda install -c conda-forge BeautifulSoup4 --yes # 
from bs4 import BeautifulSoup # import BeautifulSoup library

In [3]:
# install a parser
# !conda install -c conda-forge html5lib --yes # 

In [4]:
# install request library
# !conda install -c conda-forge request --yes
import requests

## Scrape Neighborhood Information

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text # Get the source html code
# add on .text to get the source code from response objec

In [6]:
soup = BeautifulSoup(source, 'lxml')

In [7]:
# Inspect the soup object and identify the "table" class where the desired information is located.
table = soup.find('table')

In [8]:
rowsList = [] # append rows of table
for rows in table.find_all('tr'):
    rowsList.append(rows.text)

In [9]:
pstcd = []
Borough = []
Neighborhood = []
Neighborhood_data = {'postcode':[], 'Borough':[], 'Neighborhood':[]} # neighborhood data
# rowslist includes '\n'. 
# Split using '\n' and extract the postcode, Borough and Neighborhood text for every row.
for row in rowsList:
    pstcd.append(row.split('\n')[1])
    Borough.append(row.split('\n')[2])
    Neighborhood.append(row.split('\n')[3])

# Fill dictionary    
Neighborhood_data['postcode'] = pstcd[1:]
Neighborhood_data['Borough'] = Borough[1:]
Neighborhood_data['Neighborhood'] = Neighborhood[1:]

# Make dataframe of neighborhood data using dictionary
Neighbors = pd.DataFrame(data = Neighborhood_data)

In [10]:
Neighbors.head(5)

,postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
